You can add your own songs in .mp3 format and run this cell to create your own database. Here fingerprints are extracted and saved in a hash table.

In [63]:
!python3 audfprint/audfprint.py new --dbase fingerprints.pklz audfprint/music_dataset/*.mp3


Sun Jun 30 16:35:16 2024 ingesting #0: audfprint/music_dataset/0003-Sarah.Brightman.-Time.To.Say.Goodby.mp3 ...
Sun Jun 30 16:35:17 2024 ingesting #1: audfprint/music_dataset/01 - Come Together.mp3 ...
Sun Jun 30 16:35:17 2024 ingesting #2: audfprint/music_dataset/01 - RADIOHEAD - STREET SPIRIT (FADE OUT).mp3 ...
Sun Jun 30 16:35:18 2024 ingesting #3: audfprint/music_dataset/01-Ten CC - I'm not iin love.mp3 ...
Sun Jun 30 16:35:18 2024 ingesting #4: audfprint/music_dataset/012 - The Who - Baba O'Riley.mp3 ...
Sun Jun 30 16:35:18 2024 ingesting #5: audfprint/music_dataset/02-Knack - My Sharona.mp3 ...
Sun Jun 30 16:35:19 2024 ingesting #6: audfprint/music_dataset/02. Brown sugar.mp3 ...
Sun Jun 30 16:35:19 2024 ingesting #7: audfprint/music_dataset/02. Madcon - BEGGIN'.mp3 ...
Sun Jun 30 16:35:20 2024 ingesting #8: audfprint/music_dataset/03 - SAVE ROOM.mp3 ...
Sun Jun 30 16:35:20 2024 ingesting #9: audfprint/music_dataset/03-Metallica-Master of Puppets.mp3 ...
Sun Jun 30 16:35:21 2024 

In this cell you add a youtube link and it downloads the sound in .mp3 format, it then extracts its features and compares it to the fingerprints in the database, it then returns the name of the song included in the video if it exists in the database and the duration.

In [57]:
import os
from yt_dlp import YoutubeDL

# Define the output directory and file
output_dir = 'song/'
output_file = 'song.mp3'
output_path = os.path.join(output_dir, output_file)

# Check if the file exists and delete if it does
if os.path.exists(output_path):
    os.remove(output_path)

# YouTube URL
print('Enter the youtube url:')
youtube_url = input()

# yt-dlp options
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': output_path,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'extractaudio': True,
    'audioformat': 'mp3',
}

# Download audio
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])



!python3 audfprint/audfprint.py match --dbase fingerprints.pklz song/song.mp3.mp3

Enter the youtube url:
[youtube] Extracting URL: https://www.youtube.com/watch?v=XCUZSS54drI
[youtube] XCUZSS54drI: Downloading webpage
[youtube] XCUZSS54drI: Downloading ios player API JSON
[youtube] XCUZSS54drI: Downloading m3u8 information
[info] XCUZSS54drI: Downloading 1 format(s): 251
[download] Destination: song/song.mp3
[download] 100% of    4.09MiB in 00:00:02 at 1.65MiB/s   
[ExtractAudio] Destination: song/song.mp3.mp3
Deleting original file song/song.mp3 (pass -k to keep)
Sun Jun 30 22:22:17 2024 Reading hash table fingerprints.pklz
Read fprints for 231 files ( 1377846 hashes) from fingerprints.pklz (0.21% dropped)
Sun Jun 30 22:22:18 2024 Analyzed #0 song/song.mp3.mp3 of 265.543 s to 26104 hashes
Matched song/song.mp3.mp3 265.5 sec 26104 raw hashes as audfprint/music_dataset/Bon Jovi - Have A Nice Day.mp3 at  -15.0 s with    15 of  2562 common hashes at rank  4
Processed 1 files (267.4 s total dur) in 1.2 s sec = 0.004 x RT


Here when you hit run it starts recording from your devices microphone for 5 seconds. Then it checks if the sound you played contains a song in the database and returns the title and duration as the cell above also does.

In [59]:
import os
import sounddevice as sd
import wavio

# Define the output directory and file
output_dir = os.path.expanduser('song/')
output_file = 'song.mp3.mp3'
output_path = os.path.join(output_dir, output_file)

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Check if the file exists and delete it
if os.path.exists(output_path):
    os.remove(output_path)
3
# Recording settings
duration = 5  # Duration in seconds
fs = 44100    # Sample rate
channels = 1  # Mono recording

print("Recording...")

# Record audio
audio = sd.rec(int(duration * fs), samplerate=fs, channels=channels)
sd.wait()

# Save as WAV first, then convert to MP3
wav_path = os.path.join(output_dir, 'temp.wav')
wavio.write(wav_path, audio, fs, sampwidth=2)

# Convert WAV to MP3 using ffmpeg
os.system(f'ffmpeg -i {wav_path} -codec:a libmp3lame -qscale:a 2 {output_path}')
os.remove(wav_path)

print(f"Recording saved as {output_path}")

!python3 audfprint/audfprint.py match --dbase fingerprints.pklz song/song.mp3.mp3

Recording...


ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --

Recording saved as song/song.mp3.mp3
Sun Jun 30 22:23:17 2024 Reading hash table fingerprints.pklz
Read fprints for 231 files ( 1377846 hashes) from fingerprints.pklz (0.21% dropped)
Sun Jun 30 22:23:18 2024 Analyzed #0 song/song.mp3.mp3 of 4.830 s to 533 hashes
Matched song/song.mp3.mp3 4.8 sec 533 raw hashes as audfprint/music_dataset/Marilyn Manson feat Tyler Bates - Killing Stranger - 128K MP3.mp3 at   24.1 s with    28 of    58 common hashes at rank  0
Processed 1 files (5.0 s total dur) in 0.3 s sec = 0.057 x RT


this is the inference command you can add an mp3 to /song directory manually and run this to check if it contains a song in the database

In [16]:
!python3 audfprint/audfprint.py match --dbase fingerprints.pklz song/song.mp3.mp3

Sun Jun 30 18:20:09 2024 Reading hash table fingerprints.pklz
Read fprints for 231 files ( 1377846 hashes) from fingerprints.pklz (0.21% dropped)
Sun Jun 30 18:20:09 2024 Analyzed #0 song/song.mp3.mp3 of 3.646 s to 129 hashes
Matched song/song.mp3.mp3 3.6 sec 129 raw hashes as audfprint/music_dataset/02-Knack - My Sharona.mp3 at   52.8 s with    10 of    13 common hashes at rank  0
Processed 1 files (5.0 s total dur) in 0.3 s sec = 0.063 x RT
